In [51]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,recall_score,precision_score
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import KFold
import json
from joblib import dump, load
from sklearn.model_selection import cross_val_score
from mrmr import mrmr_classif
from collections import Counter
import  matplotlib.pyplot as plt


In [52]:
data = pd.read_csv('./Data/2D_t2/all.csv')
data["Label"] = data['Label'].apply(lambda x: x-1)
data["Name"] = data['Name'].str.replace(r'^IU_', 'IUC_', regex=True)
data["Name"] = data['Name'].str.lower()

Apply T4: train = [AHN EMC IU MCA MCF NU], test=[NYU]

In [53]:
def get_results(acc_list,auc_list,recall_list,precision_list):    
    acc_list = np.array(acc_list)
    auc_list = np.array(auc_list)
    recall_list = np.array(recall_list)
    precision_list = np.array(precision_list)
    print(f'Recall, Average:{recall_list.mean():.4f}, Std:{recall_list.std():.4f}')
    print(f'precision, Average:{precision_list.mean():.4f}, Std:{precision_list.std():.4f}')
    print(f'Accuracy, Average:{acc_list.mean():.4f}, Std:{acc_list.std():.4f}')
    print(f'AUC, Average:{auc_list.mean():.4f}, Std:{auc_list.std():.4f}')
    return auc_list.mean()

In [54]:

with open('/Users/zilianghong/Documents/GitHub/IPMN-Radiomics-Plus-Deeplearning/Train_Test_4.json', 'r') as f:
    fold_data = json.load(f)

In [55]:
test_list = []
for name in fold_data['test_files']:
    test_list.append(name.split('.nii.gz')[0])
test_list = [n.lower() for n in test_list]
train_list =[]
for name in fold_data['cross_validation'][0]['train_files'] + fold_data['cross_validation'][0]['validation_files']:
    train_list.append(name.split('.nii.gz')[0])
train_list = [n.lower() for n in train_list]

In [58]:
train_data = data[data['Name'].isin(train_list)]
test_data = data[data['Name'].isin(test_list)]

In [59]:
# feature_list.sort()
feature_list = [
'skewness-Laws R5S5',
'Collage_kurt_MaximalCorrelationCoefficient_1_nb_4_ws_5',
'Collage_skew_InformationMeasureOfCorrelation2_1_nb_8_ws_3',
'Collage_kurt_Contrast_1_nb_8_ws_7',
'Collage_median_MaximalCorrelationCoefficient_1_nb_32_ws_3',
'Collage_skew_Correlation_1_nb_16_ws_5',
'Collage_skew_MaximalCorrelationCoefficient_1_nb_4_ws_3',
'Collage_skew_SumVariance_1_nb_8_ws_5',
'median-Laws E5L5',
'Collage_skew_Entropy_1_nb_16_ws_7',
'skewness-Laws S5E5',
'skewness-Laws W5L5',
'Collage_skew_DifferenceEntropy_1_nb_4_ws_3',
'Collage_var_Contrast_1_nb_8_ws_7',
'Collage_kurt_SumEntropy_1_nb_16_ws_5',
]

In [60]:
X_train = train_data.drop(columns=['Center','Name','Label'])
y_train = train_data[["Label"]]
standard_scaler = StandardScaler()
X_train_scaled = pd.DataFrame(standard_scaler.fit_transform(X_train),columns=X_train.columns)
X_train_selected = X_train_scaled[feature_list]

param_grid = {
    'n_estimators': [46,47,48,49],          
    'max_depth': [20],            
    'min_samples_split': [5],         
    'min_samples_leaf': [1],           
    'max_features': [None],  
    'bootstrap': [True]               
} 

# GridSearchCV get bset hyper parameters
rf_model = RandomForestClassifier(random_state=3)
grid_search = GridSearchCV(
    estimator=rf_model,           # 目标模型
    param_grid=param_grid,        # 参数网格
    cv=5,                         # 5折交叉验证
    scoring='roc_auc',           # 评分标准（可以选择'accuracy', 'f1'等）
    n_jobs=-1,                    # 使用所有可用内核
    verbose=1                     # 输出搜索过程
)


grid_search.fit(X_train_selected, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=3), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [20],
                         'max_features': [None], 'min_samples_leaf': [1],
                         'min_samples_split': [5],
                         'n_estimators': [46, 47, 48, 49]},
             scoring='roc_auc', verbose=1)

In [61]:
results = grid_search.cv_results_
for i in range(len(results['params'])):
    print(f"Search {i + 1}: {results['params'][i]}")
    mean_test_auc = results['mean_test_score'][i]
    print(f"Average AUC: {mean_test_auc:.4f}")
    
    for fold in range(grid_search.cv):
        # fold_test_accuracy = results[f'split{fold}_test_score'][i]
        fold_test_auc = results[f'split{fold}_test_score'][i]
        print(f"  Fold {fold + 1} - , validation AUC: {fold_test_auc:.4f}")
    print("-" * 40)

Search 1: {'bootstrap': True, 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 46}
Average AUC: 0.7807
  Fold 1 - , validation AUC: 0.7972
  Fold 2 - , validation AUC: 0.8480
  Fold 3 - , validation AUC: 0.7184
  Fold 4 - , validation AUC: 0.7664
  Fold 5 - , validation AUC: 0.7734
----------------------------------------
Search 2: {'bootstrap': True, 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 47}
Average AUC: 0.7828
  Fold 1 - , validation AUC: 0.7929
  Fold 2 - , validation AUC: 0.8505
  Fold 3 - , validation AUC: 0.7229
  Fold 4 - , validation AUC: 0.7721
  Fold 5 - , validation AUC: 0.7759
----------------------------------------
Search 3: {'bootstrap': True, 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 48}
Average AUC: 0.7849
  Fold 1 - , validation AUC: 0.7911
  Fold 2 - , validation AUC: 0.8517
  Fold 3 - , valid

Test Set

In [62]:
x_test_raw = test_data.drop(columns=['Center','Name','Label'])
y_test = test_data[["Label"]]
x_test_scaled = pd.DataFrame(standard_scaler.transform(x_test_raw),columns=x_test_raw.columns)
x_test = x_test_scaled[feature_list]

In [63]:
models = []
test_acc_list = []
test_auc_list = []
test_recall_list = []
test_precision_list = []
val_acc_list = []
val_auc_list = []
val_recall_list = []
val_precision_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=5)
for fold,(train_idx, val_idx) in enumerate(kf.split(X_train_selected, y_train)):
    X_train_f, X_val_f = X_train_selected.iloc[train_idx], X_train_selected.iloc[val_idx]
    y_train_f, y_val_f = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Get best hyper params
    best_params = grid_search.best_params_
    best_rf_model = RandomForestClassifier(**best_params, random_state=42)
    best_rf_model.fit(X_train_f, y_train_f)
    # Save model
    models.append(best_rf_model)
    
    fold_accuracy = accuracy_score(y_val_f, best_rf_model.predict(X_val_f))
    val_acc_list.append(fold_accuracy)
    fold_auc = roc_auc_score(y_val_f, best_rf_model.predict_proba(X_val_f)[:, 1])
    val_auc_list.append(fold_auc)
    print(f"Fold {fold + 1} - ACC: {fold_accuracy:.4f}, AUC: {fold_auc:.4f}")
    test_accuracy = accuracy_score(y_test, best_rf_model.predict(x_test))
    test_acc_list.append(test_accuracy)
    test_auc = roc_auc_score(y_test, best_rf_model.predict_proba(x_test)[:, 1])
    test_auc_list.append(test_auc)
    print(f"Test {fold + 1} - ACC: {test_accuracy:.4f}, AUC: {test_auc:.4f}")
    print("-" * 40)

print('Validation set')
get_results(val_acc_list,val_auc_list,val_recall_list,val_precision_list)
print("Test set")
get_results(test_acc_list,test_auc_list,test_recall_list,test_precision_list)
print('================================================')


Fold 1 - ACC: 0.7414, AUC: 0.8603
Test 1 - ACC: 0.7465, AUC: 0.7727
----------------------------------------
Fold 2 - ACC: 0.7241, AUC: 0.7604
Test 2 - ACC: 0.7042, AUC: 0.7528
----------------------------------------
Fold 3 - ACC: 0.6667, AUC: 0.7593
Test 3 - ACC: 0.6761, AUC: 0.7579
----------------------------------------
Fold 4 - ACC: 0.7719, AUC: 0.8156
Test 4 - ACC: 0.7606, AUC: 0.7959
----------------------------------------
Fold 5 - ACC: 0.5439, AUC: 0.6484
Test 5 - ACC: 0.7183, AUC: 0.7737
----------------------------------------
Validation set
Recall, Average:nan, Std:nan
precision, Average:nan, Std:nan
Accuracy, Average:0.6896, Std:0.0805
AUC, Average:0.7688, Std:0.0710
Test set
Recall, Average:nan, Std:nan
precision, Average:nan, Std:nan
Accuracy, Average:0.7211, Std:0.0301
AUC, Average:0.7706, Std:0.0151
